# Day 09. Exercise 04
# Pipelines and OOP

## 0. Imports

In [1]:
import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import joblib
from tqdm import tqdm

## 1. Preprocessing pipeline

Create three custom transformers, the first two out of which will be used within a [Pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html).

1. `FeatureExtractor()` class:
 - Takes a dataframe with `uid`, `labname`, `numTrials`, `timestamp` from the file [`checker_submits.csv`](https://drive.google.com/file/d/14voc4fNJZiLEFaZyd8nEG-lQt5JjatYw/view?usp=sharing).
 - Extracts `hour` from `timestamp`.
 - Extracts `weekday` from `timestamp` (numbers).
 - Drops the `timestamp` column.
 - Returns the new dataframe.


2. `MyOneHotEncoder()` class:
 - Takes the dataframe from the result of the previous transformation and the name of the target column.
 - Identifies all the categorical features and transforms them with `OneHotEncoder()`. If the target column is categorical too, then the transformation should not apply to it.
 - Drops the initial categorical features.
 - Returns the dataframe with the features and the series with the target column.


3. `TrainValidationTest()` class:
 - Takes `X` and `y`.
 - Returns `X_train`, `X_valid`, `X_test`, `y_train`, `y_valid`, `y_test` (`test_size=0.2`, `random_state=21`, `stratified`).


In [2]:
class FeatureExtractor(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X_copy = X.copy()
        X_copy['hour'] = pd.to_datetime(X_copy['timestamp']).dt.hour
        X_copy['weekday'] = pd.to_datetime(X_copy['timestamp']).dt.dayofweek
        X_copy = X_copy.drop('timestamp', axis=1)
        return X_copy

In [3]:
class MyOneHotEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, target_col):
        self.target_col = target_col
        self.encoder = None
        self.categorical_cols = None
    
    def fit(self, X, y=None):
        self.categorical_cols = X.select_dtypes(include=['object']).columns.tolist()
        cols_to_encode = [col for col in self.categorical_cols if col != self.target_col]
        self.categorical_cols = cols_to_encode
        
        if self.categorical_cols:
            self.encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
            self.encoder.fit(X[self.categorical_cols])
        
        return self
    
    def transform(self, X):
        X_copy = X.copy()
        
        if self.categorical_cols and self.encoder:
            encoded = self.encoder.transform(X_copy[self.categorical_cols])
            feature_names = self.encoder.get_feature_names_out(self.categorical_cols)
            encoded_df = pd.DataFrame(encoded, columns=feature_names, index=X_copy.index)
            X_copy = X_copy.drop(self.categorical_cols, axis=1)
            X_copy = pd.concat([X_copy, encoded_df], axis=1)
        
        return X_copy

In [4]:
class TrainValidationTest:
    def __init__(self, X, y, test_size=0.2, random_state=21):
        self.X = X
        self.y = y
        self.test_size = test_size
        self.random_state = random_state
    
    def split(self):
        X_temp, X_test, y_temp, y_test = train_test_split(
            self.X, self.y,
            test_size=self.test_size,
            random_state=self.random_state,
            stratify=self.y
        )
        
        X_train, X_valid, y_train, y_valid = train_test_split(
            X_temp, y_temp,
            test_size=self.test_size,
            random_state=self.random_state,
            stratify=y_temp
        )
        
        return X_train, X_valid, X_test, y_train, y_valid, y_test

## 2. Model selection pipeline

`ModelSelection()` class

 - Takes a list of `GridSearchCV` instances and a dict where the keys are the indexes from that list and the values are the names of the models, the example is below in the reverse order (from high-level to low-level perspective):

```
ModelSelection(grids, grid_dict)

grids = [gs_svm, gs_tree, gs_rf]

gs_svm = GridSearchCV(estimator=svm, param_grid=svm_params, scoring='accuracy', cv=2, n_jobs=jobs), where jobs you can specify by yourself

svm_params = [{'kernel':('linear', 'rbf', 'sigmoid'), 'C':[0.01, 0.1, 1, 1.5, 5, 10], 'gamma': ['scale', 'auto'], 'class_weight':('balanced', None), 'random_state':[21], 'probability':[True]}]
```

 - Method `choose()` takes `X_train`, `y_train`, `X_valid`, `y_valid` and returns the name of the best classifier among all the models on the validation set
 - Method `best_results()` returns a dataframe with the columns `model`, `params`, `valid_score` where the rows are the best models within each class of models.

```
model	params	valid_score
0	SVM	{'C': 10, 'class_weight': None, 'gamma': 'auto...	0.877778
1	Decision Tree	{'class_weight': 'balanced', 'criterion': 'gin...	0.866667
2	Random Forest	{'class_weight': None, 'criterion': 'entropy',...	0.907407
```

 - When you iterate through the parameters of a model class, print the name of that class and show the progress using `tqdm.notebook`, in the end of the cycle print the best model of that class.

```
Estimator: SVM
100%
125/125 [01:32<00:00, 1.36it/s]
Best params: {'C': 10, 'class_weight': None, 'gamma': 'auto', 'kernel': 'rbf', 'probability': True, 'random_state': 21}
Best training accuracy: 0.773
Validation set accuracy score for best params: 0.878 

Estimator: Decision Tree
100%
57/57 [01:07<00:00, 1.22it/s]
Best params: {'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 21, 'random_state': 21}
Best training accuracy: 0.801
Validation set accuracy score for best params: 0.867 

Estimator: Random Forest
100%
284/284 [06:47<00:00, 1.13s/it]
Best params: {'class_weight': None, 'criterion': 'entropy', 'max_depth': 22, 'n_estimators': 50, 'random_state': 21}
Best training accuracy: 0.855
Validation set accuracy score for best params: 0.907 

Classifier with best validation set accuracy: Random Forest
```

In [5]:
class ModelSelection:
    def __init__(self, grids, grid_dict):
        self.grids = grids
        self.grid_dict = grid_dict
        self.best_models = {}
        self.all_results = []
    
    def choose(self, X_train, y_train, X_valid, y_valid):
        for idx, grid in enumerate(self.grids):
            model_name = self.grid_dict[idx]
            print(f"Estimator: {model_name}")
            
            best_score = 0
            best_params = None
            best_estimator = None
            best_train_acc = 0
            
            params_list = list(self._param_combinations(grid.param_grid[0]))
            total = len(params_list)
            
            pbar = tqdm(params_list)
            
            for params in pbar:
                grid.estimator.set_params(**params)
                grid.estimator.fit(X_train, y_train)
                
                train_acc = grid.estimator.score(X_train, y_train)
                valid_acc = grid.estimator.score(X_valid, y_valid)
                
                if valid_acc > best_score:
                    best_score = valid_acc
                    best_params = params.copy()
                    best_estimator = grid.estimator
                    best_train_acc = train_acc
            
            self.best_models[model_name] = {
                'estimator': best_estimator,
                'params': best_params,
                'valid_score': best_score,
                'train_score': best_train_acc
            }
            
            self.all_results.append({
                'model': model_name,
                'params': best_params,
                'valid_score': best_score
            })
            
            print(f"100%")
            print(f"{total}/{total} [completed]")
            print(f"Best params: {best_params}")
            print(f"Best training accuracy: {best_train_acc:.3f}")
            print(f"Validation set accuracy score for best params: {best_score:.3f} \n")
        
        best_model_name = max(self.best_models.keys(), key=lambda x: self.best_models[x]['valid_score'])
        print(f"Classifier with best validation set accuracy: {best_model_name}")
        
        return best_model_name
    
    def _param_combinations(self, param_grid):
        import itertools
        keys = param_grid.keys()
        values = param_grid.values()
        
        for combination in itertools.product(*values):
            yield dict(zip(keys, combination))
    
    def best_results(self):
        return pd.DataFrame(self.all_results)

## 3. Finalization

`Finalize()` class
 - Takes an estimator.
 - Method `final_score()` takes `X_train`, `y_train`, `X_test`, `y_test` and returns the accuracy of the model as in the example below:
```
final.final_score(X_train, y_train, X_test, y_test)
Accuracy of the final model is 0.908284023668639
```
 - Method `save_model()` takes a path, saves the model to this path and prints that the model was successfully saved.

In [6]:
class Finalize:
    def __init__(self, estimator):
        self.estimator = estimator
    
    def final_score(self, X_train, y_train, X_test, y_test):
        self.estimator.fit(X_train, y_train)
        test_acc = self.estimator.score(X_test, y_test)
        print(f"Accuracy of the final model is {test_acc}")
        return test_acc
    
    def save_model(self, path):
        joblib.dump(self.estimator, path)
        print(f"Model was successfully saved")

## 4. Main program

1. Load the data from the file (****name of file****).
2. Create the preprocessing pipeline that consists of two custom transformers: `FeatureExtractor()` and `MyOneHotEncoder()`:
```
preprocessing = Pipeline([('feature_extractor', FeatureExtractor()), ('onehot_encoder', MyOneHotEncoder('dayofweek'))])
```
3. Use that pipeline and its method `fit_transform()` on the initial dataset.
```
data = preprocessing.fit_transform(df)
```
4. Get `X_train`, `X_valid`, `X_test`, `y_train`, `y_valid`, `y_test` using `TrainValidationTest()` and the result of the pipeline.
5. Create an instance of `ModelSelection()`, use the method `choose()` applying it to the models that you want and parameters that you want, get the dataframe of the best results.
6. create an instance of `Finalize()` with your best model, use method `final_score()` and save the model in the format: `name_of_the_model_{accuracy on test dataset}.sav`.

That is it, congrats!

In [7]:
df_raw = pd.read_csv('../data/checker_submits.csv')
df_target = pd.read_csv('../data/dayofweek.csv')
df_target = df_target[['dayofweek']]
df = pd.concat([df_raw, df_target], axis=1)

In [8]:
preprocessing = Pipeline([
    ('feature_extractor', FeatureExtractor()),
    ('onehot_encoder', MyOneHotEncoder('dayofweek'))
])

In [9]:
X = preprocessing.fit_transform(df)
y = X['dayofweek']
X = X.drop('dayofweek', axis=1)

In [10]:
splitter = TrainValidationTest(X, y, test_size=0.2, random_state=21)
X_train, X_valid, X_test, y_train, y_valid, y_test = splitter.split()

In [11]:
svm = SVC(kernel='rbf', random_state=21, probability=True)
svm_params = [{
    'kernel': ('linear', 'rbf', 'sigmoid'),
    'C': [0.01, 0.1, 1, 1.5, 5, 10],
    'gamma': ['scale', 'auto'],
    'class_weight': ('balanced', None),
    'probability': [True]
}]
gs_svm = GridSearchCV(estimator=svm, param_grid=svm_params, scoring='accuracy', cv=2, n_jobs=-1)

In [12]:
tree = DecisionTreeClassifier(random_state=21)
tree_params = [{
    'criterion': ('gini', 'entropy'),
    'max_depth': [5, 10, 15, 20, 21, 25],
    'class_weight': ('balanced', None)
}]
gs_tree = GridSearchCV(estimator=tree, param_grid=tree_params, scoring='accuracy', cv=2, n_jobs=-1)

In [13]:
rf = RandomForestClassifier(random_state=21)
rf_params = [{
    'n_estimators': [10, 20, 30, 40, 50],
    'criterion': ('gini', 'entropy'),
    'max_depth': [15, 20, 22, 25],
    'class_weight': ('balanced', None)
}]
gs_rf = GridSearchCV(estimator=rf, param_grid=rf_params, scoring='accuracy', cv=2, n_jobs=-1)

In [14]:
grids = [gs_svm, gs_tree, gs_rf]
grid_dict = {0: 'SVM', 1: 'Decision Tree', 2: 'Random Forest'}

In [15]:
model_selector = ModelSelection(grids, grid_dict)

In [16]:
best_model_name = model_selector.choose(X_train, y_train, X_valid, y_valid)

Estimator: SVM


100%|██████████| 72/72 [00:37<00:00,  1.91it/s]


100%
72/72 [completed]
Best params: {'kernel': 'linear', 'C': 0.1, 'gamma': 'scale', 'class_weight': 'balanced', 'probability': True}
Best training accuracy: 1.000
Validation set accuracy score for best params: 1.000 

Estimator: Decision Tree


100%|██████████| 24/24 [00:00<00:00, 243.15it/s]


100%
24/24 [completed]
Best params: {'criterion': 'gini', 'max_depth': 5, 'class_weight': 'balanced'}
Best training accuracy: 1.000
Validation set accuracy score for best params: 1.000 

Estimator: Random Forest


100%|██████████| 80/80 [00:03<00:00, 22.83it/s]

100%
80/80 [completed]
Best params: {'n_estimators': 30, 'criterion': 'entropy', 'max_depth': 20, 'class_weight': 'balanced'}
Best training accuracy: 1.000
Validation set accuracy score for best params: 1.000 

Classifier with best validation set accuracy: SVM


In [17]:
best_results_df = model_selector.best_results()
best_results_df

,model,params,valid_score
0,SVM,"{'kernel': 'linear', 'C': 0.1, 'gamma': 'scale...",1.0
1,Decision Tree,"{'criterion': 'gini', 'max_depth': 5, 'class_w...",1.0
2,Random Forest,"{'n_estimators': 30, 'criterion': 'entropy', '...",1.0


In [18]:
best_estimator = model_selector.best_models[best_model_name]['estimator']

In [19]:
finalizer = Finalize(best_estimator)

In [20]:
final_accuracy = finalizer.final_score(X_train, y_train, X_test, y_test)

Accuracy of the final model is 0.1893491124260355


In [21]:
model_filename = f'{best_model_name}_{final_accuracy:.4f}.sav'
finalizer.save_model(model_filename)

Model was successfully saved
